In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

from sklearn import tree
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib import cm
import umap

sns.set()

In [2]:
from sklearn.datasets import make_classification

## A data set with 4 features and 4 labels

In [3]:
X, y = make_classification(n_samples=500,n_features=4, n_redundant=0, n_informative=2,
                             n_clusters_per_class=1, n_classes=4)

In [4]:
reducer = umap.UMAP()

AttributeError: module 'umap' has no attribute 'UMAP'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X_embedded = reducer.fit_transform(X)

In [ ]:
plt.scatter(X_embedded[:,0],X_embedded[:,1],c=y);

In [ ]:
y

In [ ]:
X = pd.DataFrame(data = X, columns = [0,1,2,3])

In [ ]:
X

In [ ]:
from collections import Counter
C = Counter(y)
C

In [ ]:
C.most_common()

In [ ]:
C.most_common()[0][0]

In [ ]:
total = len(y)

In [ ]:
total

In [ ]:
frequencies = { k:v/total for (k,v) in C.items()}
frequencies

In [ ]:
frequencies.values()

In [ ]:
gini_index = np.sum([ p * (1 - p) for p in frequencies.values()])
gini_index

## Data set with 2 features and 4 labels

In [ ]:
Z, u = make_classification(n_features=2, n_redundant=0, n_informative=2,
                             n_clusters_per_class=1, n_classes=4)

In [ ]:
plt.scatter(Z[:,0],Z[:,1],c=u);

In [ ]:
Z = pd.DataFrame(data = Z, columns = [0,1])

In [ ]:
print(Z)

## Date set from the Visualization Notebook

In [ ]:
n = 1000
x = np.random.uniform(0, 1, n)
y = np.random.uniform(0, 1, n)
target = norm.pdf((x - 0.75) / 0.1) + norm.pdf((y - 0.75) / 0.1) \
        + norm.pdf((x - 0.25) / 0.1) + norm.pdf((y - 0.25) / 0.1) \
        + np.array(np.round(np.random.normal(-0.1,0.1, n), 2))

In [ ]:
a = plt.hist(target,bins=20)[1]

In [ ]:
def f(x):
    for i in range(len(a)):
        if x <= a[i]:
            return i
        else:
            continue   

In [ ]:
labels = np.array([f(x) for x in target])

In [ ]:
data1 = pd.DataFrame({'x' : x, 'y' : y})

### Optimal Features from Bagging notebook

In [ ]:
optim_feats = np.array(['fcf_yield', 'oiadpq', 'rd_saleq', 'market_cap', 'oancfy_q',
       'oeps12', 'ibadj12', 'book_value_yield', 'oepf12', 'quick_ratioq',
       'short_debtq', 'cf_yield', 'sic_6798', 'ibcy', 'xidoy', 'chechy',
       'inv_turnq', 'ocf_lctq', 'dpcq', 'cfmq', 'oancfy', 'roeq',
       'cfo-per-share', 'txpq', 'opmbdq', 'psq', 'dltisy', 'dltry',
       'yearly_sales', 'xsgay', 'lagppent4', 'fcf_ocfq', 'cash_ratioq',
       'de_ratioq', 'sale_equityq', 'evmq', 'opepsq', 'dvy', 'actq',
       'capxq', 'ceq4', 'fcf_csfhdq', 'sale_invcapq', 'apq', 'dlttq',
       'rect_actq', 'capeiq', 'nopiq', 'capxy', 'npmq', 'invt_actq',
       'oibdpy', 'accrualq', 'int_totdebtq', 'txtq', 'gpmq',
       'aftret_invcapxq', 'dpq'], dtype=object)

In [ ]:
len(optim_feats)

In [ ]:
max_features = 0.5
features = np.random.choice(optim_feats,int(0.5 * len(optim_feats)),replace=False)
features

In [ ]:
len(features)

In [ ]:
data_set = X
labels = y

In [ ]:
max_features = 0.5
features = np.random.choice(data_set.columns,int(0.5 * len(data_set.columns)),replace=False)
features

In [ ]:
data_set[features]

In [ ]:
f = 2
x = data_set[2].iloc[234]
x

In [ ]:
L_idxs = data_set.index[data_set[f] < x].tolist()
R_idxs = data_set.index[data_set[f] >= x].tolist()

In [ ]:
L_idxs

In [ ]:
R_idxs

In [ ]:
 L_labels = labels[L_idxs]
R_labels = labels[R_idxs]

In [ ]:
def gini_index(labels):

            C = Counter(labels)
            total = len(labels)
            frequencies = { k:v/total for (k,v) in C.items()}
            gini_index = np.sum([p*(1-p) for p in frequencies.values()])
            return gini_index

In [ ]:
print(gini_index(L_labels))
print(gini_index(R_labels))
      

In [ ]:
gini_index_of_split = (len(L_labels)*gini_index(L_labels) + len(R_labels)*gini_index(R_labels))/len(data_set)
print(gini_index_of_split)

## The RandomForests is an ensemble of Trees. Each Tree is built up of Nodes.

### Node Class

In [ ]:
class Node():
        def __init__(self,data:pd.DataFrame = None,labels=None,min_samples_leaf = 1,max_features=0.5):
            self.data = data
            self.labels = labels
            self.left = None
            self.right = None
            self.split = None
            self.max_features = max_features
            self.min_samples_leaf = min_samples_leaf
            if len(self.data) < 2 * self.min_samples_leaf:
                self.isLeaf = True
            else: self.isLeaf = False
            
        def gini_index(self,labels):

            C = Counter(labels)
            total = len(labels)
            frequencies = { k:v/total for (k,v) in C.items()}
            gini_index = np.sum([p*(1-p) for p in frequencies.values()])
            return gini_index
        
        def find_best_split(self):
            
            if self.data.empty:
                return
            
            if len(self.data) < 2*self.min_samples_leaf:
                self.isLeaf = True
                return

        
            data_set = self.data.reset_index(drop=True)
            labels = self.labels
            
            
            if self.max_features * len(data_set.columns) <= 1:
                n=1
            else:
                n = np.random.choice(range(1,int(self.max_features * len(data_set.columns))+1),1)
 
            random_features = np.random.choice(data_set.columns,n,replace=False)

            best_split_gini = np.inf
        
            best_feature = None
            best_value = None
        
            for f in random_features:
           
                for x in data_set[f]:
                
                
                    L_idxs = data_set.index[data_set[f] < x].tolist()
                    R_idxs = data_set.index[data_set[f] >= x].tolist()


                    L_labels = labels[L_idxs]
                    R_labels = labels[R_idxs]
                    
                    if (len(L_labels) < self.min_samples_leaf) or (len(R_labels) < self.min_samples_leaf):
                        continue
                        
                    else:

                        gi_L = self.gini_index(L_labels)
                        gi_R = self.gini_index(R_labels)

                        gini_index_of_split = len(L_idxs) * gi_L + len(R_idxs) * gi_R
               
                        if gini_index_of_split < best_split_gini:
                            best_split_gini = gini_index_of_split
                        
                            self.left = Node(data=data_set.iloc[L_idxs].reset_index(drop=True),
                                             labels = L_labels,min_samples_leaf = self.min_samples_leaf)
                
                            if (gi_L == 0) or (len(L_labels) < 2 * self.min_samples_leaf):
                                    self.left.isLeaf = True

                

                            self.right = Node(data = data_set.iloc[R_idxs].reset_index(drop=True),
                                              labels = R_labels,min_samples_leaf = self.min_samples_leaf)
                    
                            if (gi_R == 0) or (len(R_labels) < 2 * self.min_samples_leaf):
                                       self.right.isLeaf = True

                                
                            best_feature = f
                            best_value = x
                        
                            self.split = (best_feature,best_value)
                                
                        else: continue
  
                return
                                

### Tree class

In [ ]:
class Tree():
    
    def __init__(self,root:Node,max_depth = np.inf,min_samples_leaf=1):
        self.root = root
        self.levels = [[self.root]]
        self.max_depth = max_depth
        self.splits = []
        self.min_samples_leaf = min_samples_leaf
        self.root.min_samples_leaf = self.min_samples_leaf
        
        self.build_tree()
        
        
    def build_tree(self):
        
                while np.any([((n != None) and (not n.isLeaf)) for n in self.levels[-1]]) and (len(self.levels) < self.max_depth):
                    
                    level = []
                    
                    for node in [n for n in self.levels[-1] if ((n != None) and (not n.isLeaf))]:

                        if (node != None) and (not node.isLeaf):

                            node.find_best_split()
                            level.append(node.left)
                            level.append(node.right)
                            self.splits.append(node.split)
                        
                        else: continue
                            
                    if (len(level)==0):
                            return
                    else:
                            self.levels.append(level) 

        
    def predict(self,x):
                      
        idx = 0
        depth = len(self.levels)
        
        val = Counter(self.root.labels).most_common()[0][0]
        
        
        
        for i in range(depth):

                node = self.levels[i][idx]

                if (node.split != None):
                
                    s = node.split
                    
  
                    if (x[s[0]] < s[1]):
#                         if node.left in self.nodes[i+1]:
                            idx = self.levels[i+1].index(node.left)
                            val = Counter(node.left.labels).most_common()[0][0]
                    
                    else:
#                         if node.right in self.nodes[i+1]:
                                idx = self.levels[i+1].index(node.right)                         
                                val = Counter(node.right.labels).most_common()[0][0]  
                            
                else: return val
                
        return val


                       


## Applying the Tree to the make_classification data set

In [ ]:
root = Node(X,y)

In [ ]:
root.find_best_split()

In [ ]:
root.left.data

In [ ]:
root.labels

In [ ]:
Counter(root.labels)

In [ ]:
Counter(root.labels).most_common()

In [ ]:
Counter(root.labels).most_common()[0][0]

In [ ]:
root.split

In [ ]:
tree = Tree(root)

In [ ]:
len(tree.levels)

In [ ]:
len(tree.levels[8])

In [ ]:
tree.levels[8][9].isLeaf

In [ ]:
tree.levels

In [ ]:
tree.splits

In [ ]:
tree.levels[8][0]

In [ ]:
x = X.iloc[30].values

In [ ]:
x

In [ ]:
tree.predict(x)

In [ ]:
y[30]

### RandomForestClassifier class

In [ ]:
class RandomForestClassifier():
    
    def __init__(self,n_estimators=10,max_features=1.0,min_samples_leaf=1,bootstrap=False,max_depth = np.inf,
                 random_state=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.bootstrap = bootstrap
        if random_state is not None:
            np.random.seed(random_state)
        self.trees = []  
        
    def fit(self,data,labels):
        
        N = len(labels)
        
        for i in range(self.n_estimators):
            if self.bootstrap:
                idxs = np.random.choice(range(N),N)
                data_set = data.iloc[idxs]
                label_set = labels[idxs]
            else:
                data_set = data
                label_set = labels
                
            root = Node(data_set,label_set)
            root.max_features = self.max_features
            t = Tree(root,self.max_depth,self.min_samples_leaf)
            
            self.trees.append(t)
            
    def predict(self,data_points):
            
            if len(self.trees) == 0:
                print('Classifier needs to be fitted')
                return
            
            if type(data_points) == pd.DataFrame:
                data_points = data_points.values
            
            preds = []
            for x in data_points:

                x_preds = []
                for t in self.trees:
                    x_preds.append(t.predict(x))
                    
                preds.append(Counter(x_preds).most_common()[0][0])
                
            return preds
                
                
        
            
    

## RandomForestClassifier applied to the 2-dim make_classification data set

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=15,max_features=0.4)

In [ ]:
rf_clf.fit(Z,u)

In [ ]:
pred = rf_clf.predict(Z)

In [ ]:
plot_decision_boundary(Z.values,u,rf_clf)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(u,pred)

In [ ]:
len(u[u == pred])/len(Z)

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100,bootstrap=True,max_features=0.5,random_state=123)

In [ ]:
rf_clf.fit(Z,u)

In [ ]:
preds = rf_clf.predict(Z.values)

In [ ]:
accuracy_score(u,preds)

In [ ]:
plot_decision_boundary(Z.values,u,rf_clf)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=40,bootstrap=True,max_depth=4,max_features=0.5)

In [ ]:
rf_clf.fit(X_train,y_train)

In [ ]:
preds = rf_clf.predict(X_test)

In [ ]:
accuracy_score(y_test,preds)

In [ ]:
def plot_decision_boundary(data,labels,clf):
    plot_step = 0.02
    x_min, x_max = data[:,0].min() -1, data[:,0].max() + 1
    y_min,y_max = data[:,1].min() -1 , data[:,1].max() + 1

    xx,yy = np.meshgrid(np.arange(x_min,x_max,plot_step),
                   np.arange(y_min,y_max,plot_step))
    Z = np.array(clf.predict(np.c_[xx.ravel(),yy.ravel()]))
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx,yy,Z,cmap='cool')

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Decision Boundary')

    plt.scatter(data[:,0],data[:,1],c=labels,cmap='hot',s=100)
    plt.show()

## sklearn RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=10,bootstrap=True,max_depth=4,max_features=0.7)

In [ ]:
rf_clf.fit(X_train,y_train)

In [ ]:
preds = rf_clf.predict(X_test)

In [ ]:
accuracy_score(y_test,preds)